# TP 01 - Fondamentaux NLP pour les Transformers

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Mieux comprendre comment une machine peut interpréter du texte

---

## Comment faire interpréter du texte par une machine ?

Si vous avez déjà travaillé avec des **images** le problème est de prime abord plus simple : une image peut naturellement se découper en une grille de pixels, chaque pixel est un nombre (0-255). Le réseau peut directement les traiter.

Mais pour le **texte** suivant ?

```
"L'apprentissage automatique révolutionne l'intelligence artificielle"
```

Ce n'est qu'une suite de caractères. Un réseau de neurones, par exemple, ne comprend que des **nombres**. Comment passer de l'un à l'autre ?

---

## Les problèmes à résoudre

Pour transformer du texte en représentation numérique exploitable, il faut résoudre **deux problèmes distincts** :

### Problème 1 : La tokenization

**Comment découper le texte suivant en morceaux ?**

```
"L'apprentissage automatique" → ???
```

Plusieurs stratégies sont possibles :
- Par mots : `["L'apprentissage", "automatique"]`
- Par caractères : `["L", "'", "a", "p", "p", "r", ...]`
- Par sous-mots : `["L'", "apprent", "issage", "auto", "matique"]`

Chaque stratégie a ses avantages et inconvénients. Nous les explorerons dans ce TP.

### Problème 2 : L'embedding

**Comment transformer ces morceaux en vecteurs qui ont du SENS ?**

Une fois le texte découpé, on pourrait simplement numéroter les tokens :
```
"chat" → 42
"voiture" → 46
"chien" → 73
```

Mais ces nombres sont **arbitraires**. Ils ne capturent pas que "chat" et "chien" sont des concepts proches (animaux domestiques), alors que "voiture" est complètement différent.

**Il faut trouver un moyen** de transformer chaque token en un **vecteur de plusieurs dimensions** où la **proximité géométrique** reflète la **proximité sémantique** :

```
"chat"    → [0.2, -0.5, 0.8, ...]   ┐
                                    ├─ vecteurs proches !
"chien"   → [0.3, -0.4, 0.7, ...]   ┘

"voiture" → [-0.8, 0.2, -0.3, ...]  ← vecteur éloigné
```

Plusieurs approches existent pour construire ces vecteurs. Dans ce TP, nous explorerons **Word2Vec**, une méthode remarquable qui a révolutionné le NLP en 2013.

---

## Plan du TP

| Section | Thème | Ce que vous apprendrez |
|---------|-------|------------------------|
| §2 | Tokenization | Les 3 stratégies (mots, caractères, BPE) |
| §3 | Embeddings | Comment les vecteurs capturent le sens (Word2Vec) |
| §4 | Récapitulatif | Synthèse des concepts |
| §5 | Ressources | Pour aller plus loin |
| §6 | Attention (teaser) | Aperçu du mécanisme clé des Transformers |
| §7 | Mini-projet | Pipeline NLP complet (optionnel) |

Commençons par le premier problème : **comment découper le texte ?**

## 0. Installation et imports

Exécutez cette cellule pour installer les dépendances nécessaires.

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy gensim transformers scikit-learn datasets tokenizers -q

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# Configuration
torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

---

## 2. Tokenization : Découper le texte

La **tokenization** consiste à découper le texte en unités (tokens). Il existe plusieurs stratégies.

### 2.1 Tokenization par mots (Word-level)

La plus intuitive : on découpe sur les espaces et la ponctuation.

### Exercice 0a : Implémenter la tokenization par mots

Complétez la fonction `tokenize_words` ci-dessous. Elle doit :
- Séparer le texte sur les espaces
- Garder la ponctuation comme tokens séparés

**Indice** : Utilisez `re.findall()` pour capturer les mots OU la ponctuation.

In [ ]:
# Tokenization simple par mots
def tokenize_words(text):
    """Tokenization basique par espaces et ponctuation."""
    import re
    # Sépare sur espaces et garde la ponctuation comme tokens
    # CORRECTION: Capture les mots (\w+) OU la ponctuation ([^\w\s])
    return re.findall(r'\w+|[^\w\s]', text)

texte = "Le chat mange la souris. La souris court vite !"
tokens = tokenize_words(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème** : Le vocabulaire peut devenir énorme !

Ajoutez les noms propres, les néologismes, les mots étrangers, les fautes de frappe... Le vocabulaire explose.

À un moment, il faut **fixer une taille de vocabulaire** (ex: 50 000 mots). Mais alors, que faire des mots inconnus ?

Imaginons un vocabulaire constitué au préalable :
```
["le", "chat", "mange", ...] (50 000 mots)
```
Si un mot traité n'appartient pas au vocabulaire, l'information est perdue
```
Nouveau mot : "transformers" → <UNK> ?
```
→ Pas idéal.

### 2.2 Tokenization par caractères (Character-level)

Une solution : découper caractère par caractère. Plus de mots inconnus !

### Exercice 0b : Implémenter la tokenization par caractères

Complétez la fonction `tokenize_chars` ci-dessous.

**Indice** : En Python, une chaîne est déjà itérable caractère par caractère...

In [ ]:
def tokenize_chars(text):
    """Tokenization par caractères."""
    # CORRECTION: Une chaîne Python est itérable caractère par caractère
    return list(text)

texte = "Le chat dort."
tokens = tokenize_chars(texte)

print(f"Texte : {texte}")
print(f"Tokens : {tokens}")
print(f"Nombre de tokens : {len(tokens)}")

**Problème historique** : Séquences très longues ! "anticonstitutionnellement" = 25 tokens.

Le modèle doit "réapprendre" que `c-h-a-t` forme le concept de chat. Le coût computationnel était longtemps considéré comme prohibitif.

---

> 📚 **Des travaux récents montrent que la tokenization byte-level peut devenir compétitive**
>
> 
> **Bolmo** (Allen AI, 2025) opère directement sur les **bytes UTF-8** (256 tokens possibles) avec une architecture adaptée :
> - Un encodeur local (mLSTM) traite les bytes
> - Un "boundary predictor" regroupe les bytes en patches de taille variable
> - Le Transformer traite ces patches (pas les bytes bruts)
> 
> **Avantages** : pas de vocabulaire figé, robuste aux typos, meilleure compréhension caractère.

Pour ce TP, nous utiliserons **BPE** que nous allons voir tout de suite et qui reste le standard actuel, mais gardez en tête que le domaine évolue rapidement !

### 2.3 Tokenization Subword : BPE (Byte Pair Encoding)

Les deux approches précédentes ont des défauts :
- **Mots** : vocabulaire énorme + mots inconnus
- **Caractères** : séquences trop longues + perte de sens

**BPE** (Byte Pair Encoding) est un **compromis intelligent** utilisé par GPT, BERT, et une grande partie des LLMs modernes.

---

#### Le principe

BPE construit son vocabulaire en analysant un grand corpus de texte :

1. **Départ** : vocabulaire = tous les caractères
2. **Répéter** : trouver la paire de tokens adjacents la plus fréquente → la fusionner en un nouveau token
3. **Stop** : quand le vocabulaire atteint la taille voulue (ex: 50 000 tokens)

---

#### L'algorithme pas à pas

Pour comprendre, prenons un **corpus artificiel simplifié** :
```
"smartphone smartphone smartphone smartwatch smartwatch phone phone"
```

**Étape 0 : Partir des caractères**
```
Vocabulaire : {s, m, a, r, t, p, h, o, n, e, w, c}
```

**Étapes suivantes : Fusionner les paires les plus fréquentes**

| Étape | Paire la + fréquente | Nouveau token |
|-------|---------------------|---------------|
| 1 | (s, m) | "sm" |
| 2 | (sm, a) | "sma" |
| 3 | (sma, r) | "smar" |
| 4 | (smar, t) | "smart" |
| 5 | (p, h) | "ph" |
| 6 | (ph, o) | "pho" |
| 7 | (pho, n) | "phon" |
| 8 | (phon, e) | "phone" |

**Résultat :**
```
"smartphone" → ["smart", "phone"]  ← 2 tokens réutilisables !
"smartwatch" → ["smart", "watch"]
"phone"      → ["phone"]
```

---

#### Pourquoi c'est malin ?

Un mot **jamais vu** comme "smartcar" sera découpé en :
```
"smartcar" → ["smart", "car"]
```

Le modèle connaît déjà "smart" ! Pas besoin de token `<UNK>`.

**Bonus** : les sous-mots fréquents ont de **bons embeddings** (on verra pourquoi dans la section Word2Vec). Donc même un mot rare peut bénéficier de représentations de qualité via ses composants.

**Nous obtenons le meilleur des deux mondes** :
- Mots fréquents → tokens entiers (efficace)
- Mots rares/nouveaux → sous-mots connus (robuste)

Bien que ce modèle soit très performant dans un grand nombre de cas, il faut rester conscient que certains mots très spécifiques peuvent avoir une représentation de moindre qualité.

**Ressource** : [Explication détaillée des tokenizers (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/)

---

#### En pratique : GPT-2 vs CamemBERT

Les tokenizers BPE sont entraînés sur un corpus spécifique. **GPT-2** a été entraîné principalement sur du texte anglais, tandis que **CamemBERT** est un modèle français.

Conséquence : un même texte sera découpé différemment selon le tokenizer utilisé !

In [ ]:
# Chargement des tokenizers
from transformers import GPT2Tokenizer, CamembertTokenizer

# GPT-2 : tokenizer anglais
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# CamemBERT : tokenizer français  
camembert_tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

print("Tokenizers chargés !")
print(f"  GPT-2 : {gpt2_tokenizer.vocab_size} tokens")
print(f"  CamemBERT : {camembert_tokenizer.vocab_size} tokens")

### Exercice 0c : Comparer GPT-2 vs CamemBERT

Testez les deux tokenizers sur des phrases en français et en anglais. Observez les différences !

**Questions à explorer** :
1. Comment le tokenizer influe-t-il sur le nombre de tokens ?
2. Les tokens produits vous paraissent-ils avoir du sens ?

In [ ]:
# ============================================
# EXERCICE 0c : Comparer GPT-2 vs CamemBERT
# ============================================

# Fonction utilitaire pour comparer
def compare_tokenizers(texte):
    tokens_gpt2 = gpt2_tokenizer.tokenize(texte)
    tokens_camembert = camembert_tokenizer.tokenize(texte)
    
    print(f"Texte : '{texte}'")
    print(f"  GPT-2     : {len(tokens_gpt2):2d} tokens → {tokens_gpt2}")
    print(f"  CamemBERT : {len(tokens_camembert):2d} tokens → {tokens_camembert}")
    print()

# Exemples fournis
print("=== Phrases en FRANÇAIS ===\n")
compare_tokenizers("Le chat mange la souris.")
compare_tokenizers("L'intelligence artificielle révolutionne le monde.")
compare_tokenizers("anticonstitutionnellement")

print("=== Phrases en ANGLAIS ===\n")
compare_tokenizers("The cat eats the mouse.")
compare_tokenizers("Artificial intelligence revolutionizes the world.")
compare_tokenizers("internationalization")

# TODO: Testez vos propres phrases !
# compare_tokenizers("Votre phrase ici")

**Observations** :
- CamemBERT découpe mieux le français (moins de tokens)
- GPT-2 découpe mieux l'anglais
- Le caractère `Ġ` (GPT-2) ou `▁` (CamemBERT) indique un espace avant le token

### 2.4 Construction d'un vocabulaire

Une fois la stratégie de tokenization choisie, on construit un **vocabulaire** : une table de correspondance token ↔ index.

### Exercice 1 : Construire un vocabulaire

À partir d'un corpus, vous allez :
1. Collecter tous les tokens uniques
2. Créer un dictionnaire `vocab` avec un token spécial `<UNK>` (pour les mots inconnus)
3. Implémenter les fonctions de conversion token ↔ ID
4. Tester sur une phrase

> 💡 **Note** : En pratique, on ajoute souvent d'autres tokens spéciaux comme `<PAD>` (pour aligner les séquences de longueurs différentes lors du batching). On les verra dans les prochaines sessions.

In [ ]:
# ============================================
# EXERCICE 1 : Construire un vocabulaire
# ============================================

corpus = [
    "le chat mange",
    "le chien dort",
    "la souris court"
]

# CORRECTION 1: Collecter tous les tokens uniques du corpus
# Utilise un set() pour les tokens uniques et tokenize_words()
all_tokens = set()
for phrase in corpus:
    tokens = tokenize_words(phrase)
    all_tokens.update(tokens)


# CORRECTION 2: Créer le vocabulaire avec le token spécial <UNK>
# Le vocabulaire commence par {"<UNK>": 0}
# Puis ajoute chaque token avec un index unique
vocab = {"<UNK>": 0}
for i, token in enumerate(sorted(all_tokens), start=1):
    vocab[token] = i


# CORRECTION 3: Créer le vocabulaire inverse (id → token)
id_to_token = {idx: token for token, idx in vocab.items()}


# CORRECTION 4: Implémenter la fonction tokens_to_ids
# Retourne <UNK> (index 0) pour les mots inconnus
def tokens_to_ids(text, vocab):
    """Convertit un texte en liste d'indices."""
    tokens = tokenize_words(text)
    return [vocab.get(token, vocab["<UNK>"]) for token in tokens]


# CORRECTION 5: Implémenter la fonction ids_to_tokens
def ids_to_tokens(ids, id_to_token):
    """Convertit une liste d'indices en tokens."""
    return [id_to_token[idx] for idx in ids]


# === Tests ===
print("Vocabulaire :", vocab)
print()

# Test encodage
phrase = "le chat court"
ids = tokens_to_ids(phrase, vocab)
print(f"'{phrase}' → {ids}")

# Test décodage
tokens_back = ids_to_tokens(ids, id_to_token) if id_to_token else []
print(f"{ids} → {tokens_back}")

# Test avec mot inconnu
phrase_inconnue = "le hamster mange"
ids_inconnu = tokens_to_ids(phrase_inconnue, vocab)
phrase_inconnue_post_token = ids_to_tokens(ids_inconnu,id_to_token)
print(f"\n'{phrase_inconnue}' → {ids_inconnu} → {phrase_inconnue_post_token}")
print("(hamster devrait être remplacé par l'index de <UNK>)")

---

## 3. Embeddings

### 3.1 Le problème des indices

Les indices (0, 1, 2, ...) n'ont pas de **sens sémantique**. 

- `chat = 3` et `chien = 5` → sont-ils proches ? (oui, ce sont des animaux et ils ont beaucoup de choses en commun)
- `chat = 3` et `voiture = 4` → sont-ils proches ? (non, beaucoup moins que chien et chat)

Ces indices ne permettent pas de mesurer cette proximité !

### Approches classiques pour encoder des phrases : Bag-of-Words et TF-IDF

Historiquement, on représentait un texte par un vecteur de la taille du vocabulaire :
- **Bag-of-Words** : compter les occurrences de chaque mot
- **TF-IDF** : pondérer par la rareté des mots dans le corpus

```
Exemple simplifié (BOW vs TF-IDF) :

Corpus : ["le chat mange", "le chat dort"]
Vocabulaire : [le, chat, mange, dort]

BOW (comptage brut) :
  "le chat mange" → [1, 1, 1, 0]
  "le chat dort"  → [1, 1, 0, 1]

TF-IDF (pondéré par rareté) :
  "le chat mange" → [0.3, 0.3, 0.7, 0]    ← "mange" pèse plus (mot distinctif)
  "le chat dort"  → [0.3, 0.3, 0, 0.7]    ← "dort" pèse plus (mot distinctif)
                     ↑    ↑
              mots communs → poids réduit
```

En pratique, le vocabulaire contient 50 000+ mots → vecteurs **sparse** (majoritairement des zéros) :

```
"le chat dort" → [0, 0, ..., 1, ..., 0, 1, ..., 0]  (50 000 dimensions)
                              ↑         ↑
                            chat      dort
```

Pour TF-IDF et BOW, on obtient des vecteurs qui permettent de comparer des morceaux de texte entre eux (descriptions de produits sur un site marchand par exemple). Pour autant, ils ne permettent pas de capturer le sens des mots et leurs proximités relatives. Une méthode apparue en 2013 (Word2Vec) a permis d'apporter cette compréhension plus profonde, de manière automatisée et sans supervision.

---

### 3.2 Word2Vec : Apprendre des embeddings qui ont du sens

Comme son nom l'indique, Word2Vec transforme des mots en vecteurs. Mais comment apprend-il des vecteurs où "chat" et "chien" sont vraiment proches ?

---

#### L'intuition fondamentale

> **"Tu connais un mot par les mots qui l'entourent"** (hypothèse distributionnelle)

Observez ces phrases :
```
"Le chat mange sa pâtée"
"Le chat dort sur le canapé"  
"Mon chat joue avec une balle"

"Le chien mange sa pâtée"
"Le chien dort sur le canapé"
"Mon chien joue avec une balle"
```

"Chat" et "chien" apparaissent dans les **mêmes contextes**. Word2Vec va leur attribuer des vecteurs similaires.

---

#### L'architecture de Word2Vec

Word2Vec repose sur une architecture similaire à un réseau de neurones **étonnamment simple** : une entrée, une couche cachée, une sortie. Pas d'activation (simple multiplication matricielle).

##### Le réseau

```
      ENTRÉE                 COUCHE CACHÉE              SORTIE
     (one-hot)               (embeddings)              (softmax)
  
   ┌───┐                                              ┌───────┐
   │ 0 │  "le"                                        │ 0.02  │ "le"
   ├───┤                                              ├───────┤
   │ 0 │  "chat"             ┌───────────┐            │ 0.41  │ "chat"
   ├───┤                     │           │            ├───────┤
   │ 1 │  "noir"  ────────▶  │  vecteur  │  ────────▶ │ 0.05  │ "noir"
   ├───┤            W        │  128 dim  │     W'     ├───────┤
   │ 0 │  "dort"             │           │            │ 0.38  │ "dort"
   ├───┤                     └───────────┘            ├───────┤
   │ 0 │  "sur"                                       │ 0.14  │ "sur"
   └───┘                                              └───────┘
  
  V dimensions              D dimensions              V dimensions
  (taille vocab)            (ex: 128)                 (probabilités)
```

| Couche | Dimensions | Ce qu'elle contient |
|--------|------------|---------------------|
| Entrée | V (ex: 50 000) | Vecteur one-hot du mot |
| Cachée | D (ex: 128) | **Le vecteur qu'on veut récupérer** |
| Sortie | V (ex: 50 000) | Probabilité de chaque mot |

---

##### Le but : récupérer la couche cachée

L'objectif de Word2Vec n'est **pas** de faire des prédictions. C'est de construire de bons vecteurs.

La **matrice W** (entre l'entrée et la couche cachée) contient tous les embeddings :

```
Matrice W (V × D)
                    D dimensions
              ┌─────────────────────┐
   "le"     → │ 0.2  -0.1  0.5 ... │
   "chat"   → │ 0.3  -0.4  0.7 ... │ ← Ces deux lignes
   "chien"  → │ 0.3  -0.3  0.6 ... │ ← sont proches !
   "noir"   → │ 0.1   0.5 -0.2 ... │
   ...        │        ...         │
              └─────────────────────┘
```

**Après l'entraînement** :
- On **garde** la matrice W → c'est notre table d'embeddings
- On **jette** tout le reste (matrice W', couche de sortie)

La sortie n'était qu'un **prétexte** pour entraîner le réseau !

---

##### L'entraînement : deux techniques

Pour que les vecteurs capturent le sens des mots, on entraîne le réseau à prédire les relations entre mots voisins dans un corpus.

**Deux approches symétriques existent :**

###### Skip-gram : mot central → mots voisins

On donne un mot, le réseau prédit les mots qui l'entourent.

```
Phrase : "Le chat noir dort sur"
                  ↑
             mot central

Fenêtre de contexte (±1 mot) :
    Entrée  : "noir"
    Cibles  : "chat", "dort" (traités un par un)
```

```
    "noir"                     "chat" ?
       │                          ↑
       ▼                          │
   ┌───────┐    vecteur     ┌─────────┐
   │   W   │ ─────────────▶ │   W'    │
   └───────┘   (128 dim)    └─────────┘
```

###### CBOW : mots voisins → mot central

On donne les mots du contexte, le réseau prédit le mot du milieu.

```
Phrase : "Le chat noir dort sur"
              ↑         ↑
            contexte (±1)

Entrée  : "chat" + "dort" (moyennés)
Cible   : "noir"
```

```
"chat" + "dort"                "noir" ?
       │                          ↑
       ▼                          │
   ┌───────┐    vecteur     ┌─────────┐
   │   W   │ ─────────────▶ │   W'    │
   └───────┘   (128 dim)    └─────────┘
```

###### Comparaison

| | Skip-gram | CBOW |
|--|-----------|------|
| Entrée | 1 mot | plusieurs mots |
| Sortie | prédire les voisins | prédire le mot central |
| Mots rares | ✅ meilleur | moins bon |
| Vitesse | plus lent | ✅ plus rapide |

En pratique, **Skip-gram** est plus utilisé car il donne de meilleurs résultats sur les mots peu fréquents.

---

##### Conséquence : mots fréquents vs mots rares

Plus un mot est **fréquent** dans le corpus, plus son embedding est ajusté souvent → meilleure qualité.

Un mot vu 100 000 fois aura un excellent embedding. Un mot vu 3 fois restera proche de son initialisation aléatoire.

> C'est pour ça que BPE aide : un mot rare comme "smartcar" est découpé en "smart" + "car", deux sous-mots très fréquents avec d'excellents embeddings !

---

##### Limitation : l'ordre des mots est ignoré

Word2Vec traite chaque paire (mot, voisin) **indépendamment**. Il ne sait pas quel mot vient avant ou après.

```
"Le chat mange la souris"
"La souris mange le chat"

→ Mêmes paires d'entraînement !
→ Word2Vec ne voit pas la différence
```

C'est le **mécanisme d'attention combiné au positional encoding** (section 6) qui permettra de capturer l'ordre et les relations entre positions.

---

> 💡 **En pratique** : Les implémentations réelles (gensim, FastText...) ajoutent des optimisations pour accélérer l'entraînement sur de gros vocabulaires. L'architecture de base reste la même.

> 📚 **Pour aller plus loin** : [The Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) — Visualisations détaillées de l'architecture et de l'entraînement.

---

#### Pourquoi les analogies marchent ?

Après entraînement, les vecteurs encodent des **relations** :

```
vecteur("roi") - vecteur("homme") ≈ vecteur("reine") - vecteur("femme")
```

Autrement dit, la "direction" homme→femme dans l'espace vectoriel est la même que roi→reine :

```
        homme ─────────────→ femme
          ↑    (même         ↑
          │   direction)     │
         roi ─────────────→ reine
```

C'est pour ça que `roi - homme + femme ≈ reine` fonctionne !

---

#### Mesurer la similarité : Similarité cosinus

Un vecteur possède **deux caractéristiques** :
- **Une direction** (où il pointe)
- **Une norme** (sa longueur)

A priori, pour comparer deux vecteurs, on devrait s'intéresser aux deux.

##### Le cas de Word2Vec

En pratique, la norme des embeddings Word2Vec est **polluée** par des effets qui n'ont rien à voir avec le sens : fréquence des mots dans le corpus, détails de l'entraînement...

On utilise donc la **similarité cosinus**, qui compare uniquement les directions. Et ça fonctionne très bien !

$$\text{similarité}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

| Valeur | Interprétation |
|--------|----------------|
| 1 | Même direction |
| 0 | Aucune relation |
| -1 | Directions opposées |

##### Les modèles récents

Les modèles modernes (OpenAI, Sentence-Transformers...) intègrent la normalisation **directement dans l'entraînement** et retournent des embeddings déjà normalisés. Résultat : cosine similarity = dot product, plus d'ambiguïté.

> 📚 **Pour aller plus loin** :
> - [Pinecone - Vector Similarity Explained](https://www.pinecone.io/learn/vector-similarity/)
> - [ArXiv - Is Cosine-Similarity Really About Similarity?](https://arxiv.org/abs/2403.05440) — Limites théoriques

---

### 3.3 Exploration avec GloVe

Nous allons utiliser **GloVe** (similaire à Word2Vec), entraîné sur Wikipedia.

In [ ]:
# Charger un modèle pré-entraîné (GloVe, similaire à Word2Vec)
import gensim.downloader as api

print("Chargement du modèle GloVe (peut prendre 1-2 min)...")
model = api.load("glove-wiki-gigaword-100")  # 100 dimensions, entraîné sur Wikipedia
print(f"Modèle chargé ! Vocabulaire : {len(model)} mots")

In [ ]:
# Explorer les similarités
print("=== Mots similaires à 'king' ===")
for word, score in model.most_similar("king", topn=5):
    print(f"  {word}: {score:.4f}")

print("\n=== Mots similaires à 'computer' ===")
for word, score in model.most_similar("computer", topn=5):
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 2 : Explorer les similarités
# ============================================

# Partie A : Trouver les mots similaires à "france", "cat", "happy"
# TODO: Utilisez model.most_similar(mot, topn=5)


# Partie B : Exploration libre
# TODO: Trouvez une paire de mots avec similarité > 0.7
# Indice : model.similarity("mot1", "mot2") retourne un score entre -1 et 1


# Partie C : Trouver l'intrus
# TODO: Utilisez model.doesnt_match(["mot1", "mot2", "mot3", "mot4"])
# Exemple : model.doesnt_match(["breakfast", "lunch", "dinner", "car"])
# Testez avec vos propres listes !

In [ ]:
# La magie des analogies : king - man + woman = ?
print("=== Analogie : king - man + woman = ? ===")
result = model.most_similar(positive=["king", "woman"], negative=["man"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

print("\n=== Analogie : paris - france + italy = ? ===")
result = model.most_similar(positive=["paris", "italy"], negative=["france"], topn=3)
for word, score in result:
    print(f"  {word}: {score:.4f}")

In [ ]:
# ============================================
# EXERCICE 3 : Maîtriser les analogies
# ============================================

# Partie A : Tester ces analogies classiques
# - "berlin" - "germany" + "france" = ?
# - "good" - "better" + "bad" = ?
# - "walked" - "walk" + "swim" = ?

# TODO: Syntaxe : model.most_similar(positive=["A", "C"], negative=["B"], topn=3)


# Partie B : Inventer une analogie qui fonctionne
# TODO: Trouvez une analogie originale qui donne le résultat attendu
# Exemples de domaines : métiers, pays/capitales, animaux, verbes...


# Partie C : Trouver une analogie qui échoue
# TODO: Trouvez une analogie qui devrait marcher logiquement mais échoue
# Expliquez pourquoi dans un commentaire (indice : fréquence des mots, biais du corpus...)

In [ ]:
# Visualisation des embeddings en 2D
from sklearn.decomposition import PCA

# Sélectionner quelques mots
words = ["king", "queen", "man", "woman", "prince", "princess",
         "cat", "dog", "lion", "tiger",
         "car", "bus", "train", "plane"]

# Récupérer leurs vecteurs
vectors = np.array([model[w] for w in words])

# Réduire à 2D avec PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(vectors)

# Visualiser
plt.figure(figsize=(12, 8))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='blue', s=100)

for i, word in enumerate(words):
    plt.annotate(word, (vectors_2d[i, 0] + 0.1, vectors_2d[i, 1] + 0.1), fontsize=12)

plt.title("Embeddings GloVe projetés en 2D")
plt.xlabel("Composante 1")
plt.ylabel("Composante 2")
plt.grid(True, alpha=0.3)
plt.show()

print("Observation : Les mots de même catégorie sont regroupés !")

In [ ]:
# ============================================
# EXERCICE 4 : Visualisation personnalisée
# ============================================

# TODO: Créez votre propre visualisation avec 15-20 mots de votre choix
# Choisissez des mots de 3-4 catégories différentes (ex: sports, émotions, pays, métiers)

# Vos mots (modifiez cette liste) :
my_words = [
    # Catégorie 1 (ex: sports) : 
    
    # Catégorie 2 (ex: émotions) : 
    
    # Catégorie 3 (ex: pays) : 
    
    # Catégorie 4 (ex: métiers) : 
]

# TODO: Vérifiez que tous vos mots sont dans le vocabulaire
# for word in my_words:
#     if word not in model:
#         print(f"'{word}' n'est pas dans le vocabulaire !")

# TODO: Copiez et adaptez le code de visualisation de la cellule précédente
# Remplacez 'words' par 'my_words'


# QUESTION : Les mots de même catégorie sont-ils regroupés ? 
# Y a-t-il des surprises ?

---

## 4. Récapitulatif

### Le pipeline NLP

| Étape | Entrée | Sortie | Rôle |
|-------|--------|--------|------|
| **Tokenization** | Texte brut | Liste de tokens | Découper le texte |
| **Vocabulaire** | Tokens | Indices | Table token ↔ ID |
| **Embedding** | Indices | Vecteurs denses | Sens sémantique |

### Points clés

1. **Tokenization BPE** : meilleur compromis entre mots et caractères
2. **Embeddings** : transforment les mots en vecteurs comparables
3. **Word2Vec** : montre que les embeddings capturent le sens (analogies !)
4. **Similarité cosinus** : mesure la proximité entre vecteurs

### Prochaine session

Nous verrons le **mécanisme d'attention** : comment les mots "communiquent" entre eux pour se comprendre mutuellement.

---

## 5. Pour aller plus loin (optionnel)

### Ressources

- [Les tokenizers en NLP (FR)](https://lbourdois.github.io/blog/nlp/Les-tokenizers/) - Excellent article en français
- [The Illustrated Word2Vec](https://jalammar.github.io/illustrated-word2vec/) - Visualisations très claires
- [Bolmo: Byte-level Language Models (Allen AI, 2025)](https://allenai.org/blog/bolmo) - Une alternative à BPE ?

### Expérimentations suggérées

1. Tester d'autres analogies Word2Vec
2. Visualiser les embeddings de votre choix
3. Comparer différents tokenizers (BERT, GPT-2, etc.)

---

## 6. Teaser : Le mécanisme d'attention

Maintenant que nous savons représenter du texte (tokenization + embeddings), la prochaine étape est de permettre aux mots de **communiquer entre eux**.

C'est le rôle du **mécanisme d'attention**, que nous verrons en détail au prochain TP.

### L'idée

> Pour comprendre un mot, il faut regarder les autres mots de la phrase.

Exemple : *"Le chat qui dormait sur le canapé a sauté"*
- Pour comprendre **"a sauté"** → regarder **"chat"** (le sujet)
- Pour comprendre **"dormait"** → regarder **"chat"** et **"canapé"**

### Visualisation

In [ ]:
# Matrice d'attention simulée
phrase = ["Le", "chat", "mange", "la", "souris"]

# Chaque ligne = un mot qui "regarde" les autres
# Valeurs = poids d'attention (somme = 1 par ligne)
attention = torch.tensor([
    [0.8, 0.1, 0.05, 0.03, 0.02],  # "Le" regarde surtout lui-même
    [0.1, 0.7, 0.1, 0.05, 0.05],   # "chat" regarde surtout lui-même
    [0.05, 0.4, 0.4, 0.05, 0.1],   # "mange" regarde "chat" et lui-même
    [0.02, 0.03, 0.05, 0.8, 0.1],  # "la" regarde surtout lui-même
    [0.02, 0.1, 0.2, 0.08, 0.6],   # "souris" regarde "mange" et elle-même
])

# Visualisation
plt.figure(figsize=(8, 6))
plt.imshow(attention, cmap='Blues')
plt.xticks(range(5), phrase)
plt.yticks(range(5), phrase)
plt.xlabel("Mots regardés")
plt.ylabel("Mots qui regardent")
plt.title("Qui regarde qui ? (Matrice d'attention)")
plt.colorbar(label="Poids d'attention")

for i in range(5):
    for j in range(5):
        plt.text(j, i, f'{attention[i,j]:.2f}', 
                ha='center', va='center',
                color='white' if attention[i,j] > 0.5 else 'black')
plt.show()

print("Le verbe 'mange' regarde fortement 'chat' (son sujet) !")

**Ce qu'on voit** :
- Chaque mot peut "regarder" tous les autres mots
- Les poids indiquent l'importance de chaque relation
- Le modèle **apprend** ces poids pendant l'entraînement

**Au prochain TP**, nous verrons :
- Comment calculer cette matrice d'attention
- Les concepts Query, Key, Value
- L'architecture complète du Transformer

---

## 7. Mini-projet : Construire son propre pipeline NLP

> **🏠 BONUS À FAIRE CHEZ SOI**
> 
> Cette section est **optionnelle** et ne fait pas partie du TP en session.
> Elle est proposée pour les étudiants qui souhaitent approfondir à la maison.

Dans ce projet, vous allez **tout construire à partir de libs génériques** : votre propre tokenizer BPE et vos propres embeddings Word2Vec, entraînés sur un corpus thématique Pokémon !

### Objectifs

1. **Entraîner un tokenizer BPE** adapté au vocabulaire Pokémon
2. **Entraîner Word2Vec** sur les tokens BPE
3. **Explorer les similarités** entre Pokémon, types, attaques...
4. **(Avancé)** Comparer avec un modèle français pré-entraîné fine-tuné

### Structure du projet

| Partie | Contenu | Difficulté |
|--------|---------|------------|
| **Partie 1** | BPE custom + Word2Vec from scratch | ⭐⭐ |
| **Partie 2** | Fine-tuning FastText français | ⭐⭐⭐ |
| **Partie 3** | Comparaison des deux approches | ⭐ |

---

### Partie 1 : Pipeline from scratch

#### 1.1 Chargement du corpus Pokémon

Nous utilisons un corpus extrait de **Poképedia** (le wiki Pokémon francophone) contenant les descriptions de Pokémon, attaques, lieux, et bien plus.

In [ ]:
# ============================================
# PARTIE 1.1 : Chargement du corpus Pokémon
# ============================================

from datasets import load_dataset

# Charger le corpus Pokémon depuis Hugging Face
dataset = load_dataset("chris-lmd/pokepedia-fr")

print(f"Nombre d'articles : {len(dataset['train'])}")

# Extraire le texte
corpus_texts = [article["content"] for article in dataset["train"]]

# Aperçu
print(f"\n=== Exemple d'article ===")
print(f"Titre : {dataset['train'][0]['title']}")
print(f"Contenu (extrait) :\n{corpus_texts[0][:500]}...")

# Statistiques
total_chars = sum(len(t) for t in corpus_texts)
total_words = sum(len(t.split()) for t in corpus_texts)
print(f"\n=== Statistiques du corpus ===")
print(f"Nombre d'articles : {len(corpus_texts)}")
print(f"Nombre total de mots : {total_words:,}")
print(f"Nombre total de caractères : {total_chars:,}")

#### 1.2 Entraînement du tokenizer BPE

Nous allons créer notre propre tokenizer BPE adapté au vocabulaire Pokémon. La bibliothèque `tokenizers` de Hugging Face permet d'entraîner un BPE en quelques lignes.

**Rappel** : BPE fusionne itérativement les paires de caractères les plus fréquentes. Sur notre corpus, il apprendra des tokens comme :
- `"Pika"` + `"chu"` → fréquent ensemble
- `"Draco"` + `"feu"` → fusion possible
- `"évolu"` + `"tion"` → motif récurrent

In [ ]:
# ============================================
# PARTIE 1.2 : Entraînement du tokenizer BPE
# ============================================

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from collections import Counter

# === Étape 0 : Analyser les mots uniques du corpus ===
print("Analyse du corpus...")
all_words = []
for text in corpus_texts:
    # Tokenization simple par espaces (mots bruts)
    words = text.split()
    all_words.extend(words)

word_counts = Counter(all_words)
unique_words = set(all_words)

print(f"  Nombre total de mots : {len(all_words):,}")
print(f"  Mots uniques : {len(unique_words):,}")
print(f"  Top 10 mots : {word_counts.most_common(10)}")

# === Étape 1 : Créer et entraîner le tokenizer BPE ===
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# Taille du vocabulaire - ajustable !
VOCAB_SIZE = 8000

trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[UNK]", "[PAD]"],
    show_progress=True
)

print(f"\nEntraînement du tokenizer BPE (vocab_size={VOCAB_SIZE})...")
tokenizer.train_from_iterator(corpus_texts, trainer=trainer)
print(f"Tokenizer entraîné ! Vocabulaire : {tokenizer.get_vocab_size()} tokens")

# === Étape 2 : Analyser la couverture ===
print("\n" + "=" * 50)
print("ANALYSE DE LA COUVERTURE")
print("=" * 50)

# Compter combien de mots uniques = 1 seul token
words_as_single_token = []
words_split = []

for word in unique_words:
    tokens = tokenizer.encode(word).tokens
    if len(tokens) == 1 and tokens[0] != "[UNK]":
        words_as_single_token.append(word)
    else:
        words_split.append((word, tokens))

coverage = len(words_as_single_token) / len(unique_words) * 100

print(f"\nMots uniques du corpus : {len(unique_words):,}")
print(f"Mots = 1 token (couverts) : {len(words_as_single_token):,} ({coverage:.1f}%)")
print(f"Mots découpés en 2+ tokens : {len(words_split):,} ({100-coverage:.1f}%)")

print(f"\n📊 Ratio vocab_size / mots_uniques : {VOCAB_SIZE / len(unique_words):.1%}")

# Exemples
print("\n✅ Exemples de mots couverts (1 token) :")
sample_covered = [w for w in words_as_single_token if len(w) > 4][:10]
print(f"  {sample_covered}")

print("\n❌ Exemples de mots découpés :")
sample_split = [(w, t) for w, t in words_split if len(w) > 5 and len(t) <= 4][:5]
for word, tokens in sample_split:
    print(f"  '{word}' → {tokens}")

# === Étape 3 : Test du tokenizer ===
print("\n" + "=" * 50)
print("TEST DU TOKENIZER")
print("=" * 50)

test_phrases = [
    "Pikachu utilise Tonnerre",
    "Dracaufeu est un Pokémon de type Feu",
    "Évolution de Salamèche en Reptincel",
    "Méga-Évolution disponible"
]

for phrase in test_phrases:
    output = tokenizer.encode(phrase)
    print(f"'{phrase}'")
    print(f"  → {output.tokens}")
    print()

#### Comment interpréter ces résultats ?

> **Pourquoi seulement ~4% de couverture ?**
> 
> Ce chiffre peut sembler faible, mais c'est **normal et attendu** :
> 
> 1. **BPE optimise pour la fréquence** : Les mots très fréquents deviennent des tokens entiers, les mots rares sont découpés en sous-mots connus
> 
> 2. **107k mots uniques** inclut beaucoup de "bruit" : hapax (mots vus 1 seule fois), mots avec ponctuation collée, fautes de frappe, mots étrangers...
> 
> 3. **L'important** : Les mots du domaine (Pikachu, Dracaufeu, évolution, attaque...) sont bien des tokens entiers
> 
> **Règle pratique** :
> - Si les mots **importants** de votre domaine sont découpés → augmenter `vocab_size`
> - Si beaucoup de tokens n'apparaissent que 1-2 fois → réduire `vocab_size`
> 
> **Ordres de grandeur** :
> | Modèle | vocab_size | Corpus |
> |--------|------------|--------|
> | GPT-2 | 50k | ~40 Go |
> | CamemBERT | 32k | ~138 Go |
> | Notre corpus | 8k | ~10 Mo |

#### 1.3 Tokenization du corpus

Maintenant, appliquons notre tokenizer BPE à tout le corpus pour préparer l'entraînement de Word2Vec.

In [ ]:
# ============================================
# PARTIE 1.3 : Tokenization du corpus complet
# ============================================

import random

# Tokenizer tout le corpus
print("Tokenization du corpus...")
corpus_tokenized = []

for i, text in enumerate(corpus_texts):
    # Découper le texte en phrases (approximatif)
    sentences = text.replace('\n', ' ').split('. ')
    
    for sentence in sentences:
        if len(sentence.strip()) > 10:  # Ignorer les phrases trop courtes
            output = tokenizer.encode(sentence)
            tokens = output.tokens
            if len(tokens) >= 3:  # Au moins 3 tokens pour Word2Vec
                corpus_tokenized.append(tokens)
    
    if (i + 1) % 1000 == 0:
        print(f"  {i + 1}/{len(corpus_texts)} articles traités...")

print(f"\nCorpus tokenisé !")
print(f"Nombre de phrases : {len(corpus_tokenized):,}")
print(f"Nombre total de tokens : {sum(len(s) for s in corpus_tokenized):,}")

# Exemples aléatoires (phrases assez longues)
print("\n=== Exemples de phrases tokenisées ===")
long_phrases = [p for p in corpus_tokenized if len(p) >= 12]
for phrase in random.sample(long_phrases, min(5, len(long_phrases))):
    print(f"  {phrase[:12]}...")

#### 1.4 Entraînement de Word2Vec

Nous utilisons **gensim** pour entraîner Word2Vec sur notre corpus tokenisé.

**Paramètres importants** :
- `vector_size` : dimension des embeddings (100-300 typique)
- `window` : taille de la fenêtre de contexte
- `min_count` : ignorer les tokens trop rares
- `sg=1` : utiliser Skip-gram (meilleur pour les mots rares)

In [ ]:
# ============================================
# PARTIE 1.4 : Entraînement de Word2Vec
# ============================================

from gensim.models import Word2Vec

print("Entraînement de Word2Vec...")

# Entraîner le modèle
model_w2v = Word2Vec(
    sentences=corpus_tokenized,
    vector_size=100,      # Dimension des embeddings
    window=5,             # Fenêtre de contexte (±5 mots)
    min_count=3,          # Ignorer les tokens vus < 3 fois
    sg=1,                 # Skip-gram (1) vs CBOW (0)
    workers=4,            # Parallélisation
    epochs=10             # Nombre de passes sur le corpus
)

print(f"Modèle entraîné !")
print(f"Vocabulaire Word2Vec : {len(model_w2v.wv)} tokens")
print(f"Dimension des vecteurs : {model_w2v.wv.vector_size}")

# Sauvegarder le modèle (optionnel)
# model_w2v.save("pokemon_word2vec.model")

#### 1.5 Exploration des embeddings Pokémon

C'est le moment de tester si notre modèle a capturé les relations sémantiques du monde Pokémon !

In [ ]:
# ============================================
# PARTIE 1.5 : Exploration des similarités
# ============================================

# Fonction utilitaire pour chercher un token
def find_token(word, model):
    """Cherche un token dans le vocabulaire (insensible à la casse)."""
    word_lower = word.lower()
    for token in model.wv.key_to_index:
        if word_lower == token.lower():
            return token
    return None

# Fonction pour afficher les similarités
def show_similar(word, model, topn=10):
    """Affiche les mots les plus similaires."""
    token = find_token(word, model)
    if token is None:
        print(f"'{word}' non trouvé dans le vocabulaire")
        return
    
    print(f"=== Mots similaires à '{token}' ===")
    try:
        for similar, score in model.wv.most_similar(token, topn=topn):
            print(f"  {similar}: {score:.4f}")
    except KeyError as e:
        print(f"Erreur: {e}")

# Tester avec des noms de Pokémon
print("=" * 50)
show_similar("Pikachu", model_w2v)
print()
show_similar("Dracaufeu", model_w2v)
print()
show_similar("Évolution", model_w2v)

In [ ]:
# ============================================
# EXERCICE 5 : Explorer votre modèle Pokémon
# ============================================

# TODO A : Testez les similarités avec d'autres Pokémon
# Exemples : Salamèche, Carapuce, Bulbizarre, Mewtwo, Rondoudou...
# show_similar("...", model_w2v)


# TODO B : Testez avec des types (Feu, Eau, Plante, Électrik...)
# show_similar("Feu", model_w2v)


# TODO C : Testez avec des attaques (Tonnerre, Lance-Flammes, Surf...)
# show_similar("Tonnerre", model_w2v)


# TODO D : Essayez des analogies Pokémon !
# Exemple : Pikachu - Électrik + Feu = ?
# model_w2v.wv.most_similar(positive=["Pikachu", "Feu"], negative=["Électrik"], topn=5)


# QUESTIONS DE RÉFLEXION :
# 1. Les Pokémon de même type sont-ils proches ?
# 2. Les évolutions sont-elles proches (Salamèche ↔ Dracaufeu) ?
# 3. Quelles analogies fonctionnent ? Lesquelles échouent ?

In [ ]:
# ============================================
# PARTIE 1.6 : Visualisation des embeddings
# ============================================

from sklearn.decomposition import PCA

# Sélectionner des tokens intéressants (à adapter selon votre vocabulaire)
# Vérifiez d'abord quels tokens existent :
print("=== Quelques tokens du vocabulaire ===")
vocab_sample = list(model_w2v.wv.key_to_index.keys())[:50]
print(vocab_sample)

# TODO: Sélectionnez des tokens pour la visualisation
# Choisissez des Pokémon, types, attaques...
tokens_to_plot = [
    # Pokémon (vérifiez qu'ils existent dans vocab_sample ou cherchez-les)
    # "Pikachu", "Raichu", ...
    
    # Types
    # "Feu", "Eau", ...
    
    # Attaques
    # "Tonnerre", ...
]

# Filtrer les tokens qui existent dans le vocabulaire
tokens_valid = [t for t in tokens_to_plot if t in model_w2v.wv]
print(f"\nTokens valides : {len(tokens_valid)}/{len(tokens_to_plot)}")

if len(tokens_valid) >= 5:
    # Récupérer les vecteurs
    vectors = np.array([model_w2v.wv[t] for t in tokens_valid])
    
    # Réduire à 2D
    pca = PCA(n_components=2)
    vectors_2d = pca.fit_transform(vectors)
    
    # Visualiser
    plt.figure(figsize=(14, 10))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='red', s=100)
    
    for i, token in enumerate(tokens_valid):
        plt.annotate(token, (vectors_2d[i, 0] + 0.02, vectors_2d[i, 1] + 0.02), fontsize=10)
    
    plt.title("Embeddings Pokémon (notre modèle) projetés en 2D")
    plt.xlabel("Composante 1")
    plt.ylabel("Composante 2")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Pas assez de tokens valides pour la visualisation.")

---

### Partie 2 : Fine-tuning FastText français (Bonus ⭐⭐⭐)

> ⚠️ **Attention : Ressources requises**
> 
> Cette partie nécessite **~8 Go de RAM** pour charger le modèle FastText français.
> - **Google Colab gratuit** (12.7 Go) : peut fonctionner mais risque de saturation mémoire
> - **Google Colab Pro** (25+ Go) : recommandé
> - **Machine locale** : 16 Go RAM minimum
> 
> Si vous rencontrez une erreur mémoire, passez directement à la **Partie 3** qui compare votre modèle Word2Vec avec les concepts théoriques.

Dans cette partie, nous allons **fine-tuner** un modèle FastText pré-entraîné sur le français, puis l'enrichir avec notre corpus Pokémon.

**Différence avec la Partie 1** :
- Partie 1 : On part de zéro → le modèle ne connaît QUE le monde Pokémon
- Partie 2 : On part d'un modèle français → il connaît déjà la langue + on ajoute Pokémon

**Avantage de FastText** : Il gère les **sous-mots**, donc même un mot jamais vu comme "Méga-Dracaufeu" sera partiellement compris.

In [ ]:
# ============================================
# PARTIE 2.1 : Téléchargement du modèle FastText français
# ============================================

import os
from gensim.models.fasttext import load_facebook_model

# Le modèle FastText français officiel (~1.2 Go compressé, ~7 Go en mémoire)
# Source : https://fasttext.cc/docs/en/crawl-vectors.html

MODEL_PATH = "cc.fr.300.bin"
MODEL_GZ_PATH = "cc.fr.300.bin.gz"

# Télécharger le modèle français (si pas déjà fait)
if not os.path.exists(MODEL_PATH):
    if not os.path.exists(MODEL_GZ_PATH):
        print("📥 Téléchargement du modèle FastText français (~1.2 Go)...")
        print("⚠️ Cela peut prendre 5-10 minutes selon votre connexion.\n")
        !wget -q --show-progress https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
    
    print("\n📦 Décompression du modèle...")
    !gunzip -k cc.fr.300.bin.gz
    print("✅ Décompression terminée !")
else:
    print(f"✅ Modèle déjà présent : {MODEL_PATH}")

# Charger le modèle avec gensim (permet le fine-tuning !)
print("\n🔄 Chargement du modèle FastText français...")
print("(Cela peut prendre 1-2 minutes et utiliser ~7 Go de RAM)")

model_fasttext_fr = load_facebook_model(MODEL_PATH)

print(f"\n✅ Modèle chargé !")
print(f"   Vocabulaire : {len(model_fasttext_fr.wv):,} mots")
print(f"   Dimension : {model_fasttext_fr.wv.vector_size}")

# Test rapide du modèle français
print("\n=== Test du modèle français ===")
test_words_fr = ["chat", "chien", "voiture", "ordinateur", "maison"]
for word in test_words_fr:
    try:
        similar = model_fasttext_fr.wv.most_similar(word, topn=3)
        print(f"'{word}' → {[w for w, s in similar]}")
    except KeyError:
        print(f"'{word}' → non trouvé")

In [ ]:
# ============================================
# PARTIE 2.2 : Fine-tuning sur le corpus Pokémon
# ============================================

# Avec gensim, on peut continuer l'entraînement d'un modèle FastText
# en ajoutant de nouveaux mots au vocabulaire puis en réentraînant

print("📊 Préparation du corpus pour le fine-tuning...")

# Préparer le corpus (liste de listes de mots)
corpus_for_finetuning = []
for text in corpus_texts:
    sentences = text.replace('\n', ' ').split('. ')
    for sentence in sentences:
        words = sentence.strip().split()
        if len(words) >= 3:  # Au moins 3 mots par phrase
            corpus_for_finetuning.append(words)

print(f"   Phrases préparées : {len(corpus_for_finetuning):,}")
print(f"   Tokens totaux : {sum(len(s) for s in corpus_for_finetuning):,}")

# Vocabulaire avant fine-tuning
vocab_before = len(model_fasttext_fr.wv)

# Étendre le vocabulaire avec les mots Pokémon
print("\n🔧 Extension du vocabulaire avec les mots Pokémon...")
model_fasttext_fr.build_vocab(corpus_for_finetuning, update=True)

vocab_after = len(model_fasttext_fr.wv)
new_words = vocab_after - vocab_before
print(f"   Vocabulaire avant : {vocab_before:,} mots")
print(f"   Vocabulaire après : {vocab_after:,} mots")
print(f"   Nouveaux mots ajoutés : {new_words:,}")

# Fine-tuning : continuer l'entraînement sur le corpus Pokémon
print("\n🚀 Fine-tuning en cours (5 epochs)...")
print("   Cela peut prendre quelques minutes...")

model_fasttext_fr.train(
    corpus_for_finetuning,
    total_examples=len(corpus_for_finetuning),
    epochs=5
)

print("\n✅ Fine-tuning terminé !")

# Test avec des mots Pokémon
print("\n=== Test après fine-tuning ===")
test_pokemon = ["Pikachu", "Dracaufeu", "évolution", "attaque", "Pokémon"]
for word in test_pokemon:
    try:
        similar = model_fasttext_fr.wv.most_similar(word, topn=5)
        print(f"'{word}' →")
        for w, score in similar:
            print(f"    {w}: {score:.3f}")
        print()
    except KeyError:
        print(f"'{word}' → non trouvé\n")

---

### Partie 3 : Comparaison des approches

Comparons maintenant les deux modèles (si vous avez fait la Partie 2) ou analysons les forces et faiblesses de notre modèle from scratch.

| Critère | BPE + Word2Vec (Partie 1) | FastText fine-tuné (Partie 2) |
|---------|---------------------------|------------------------------|
| **Vocabulaire** | Tokens BPE Pokémon | Mots français + Pokémon |
| **Mots inconnus** | `[UNK]` | Géré par sous-mots |
| **Mots FR courants** | Qualité variable | Excellente (pré-entraîné) |
| **Mots Pokémon rares** | Dépend de la fréquence | Sous-mots aident |
| **Taille modèle** | ~10-50 Mo | ~1.2 Go |
| **Temps d'entraînement** | Rapide | Plus long |

In [ ]:
# ============================================
# PARTIE 3 : Tests comparatifs
# ============================================

# Vérifier si le modèle FastText a été chargé (Partie 2)
fasttext_available = 'model_fasttext_fr' in dir() and model_fasttext_fr is not None

# Tests à effectuer
test_words = [
    # Pokémon populaires
    "Pikachu", "Dracaufeu", "Mewtwo",
    # Types
    "Feu", "Eau", "Électrik",
    # Mots du domaine Pokémon
    "attaque", "combat", "évolution",
    # Mots français généraux (test du pré-entraînement)
    "ordinateur", "maison", "voiture"
]

print("=" * 60)
print("COMPARAISON DES MODÈLES")
print("=" * 60)

print("\n📦 Modèle 1 : BPE + Word2Vec (from scratch)")
print("-" * 40)
for word in test_words:
    token = find_token(word, model_w2v)
    if token:
        print(f"  ✅ {word} → '{token}'")
    else:
        print(f"  ❌ {word} → non trouvé")

if fasttext_available:
    print("\n📦 Modèle 2 : FastText français fine-tuné")
    print("-" * 40)
    for word in test_words:
        try:
            vec = model_fasttext_fr.wv[word]
            in_vocab = word in model_fasttext_fr.wv.key_to_index
            status = "✅" if in_vocab else "🔶"
            print(f"  {status} {word} → vecteur dim {len(vec)}" + 
                  (" (inféré)" if not in_vocab else ""))
        except:
            print(f"  ❌ {word} → erreur")

    # Comparaison des similarités
    print("\n" + "=" * 60)
    print("COMPARAISON DES SIMILARITÉS")
    print("=" * 60)

    comparison_words = ["Pikachu", "évolution", "maison"]

    for word in comparison_words:
        print(f"\n🔍 Mots similaires à '{word}' :")
        
        # Modèle 1
        token = find_token(word, model_w2v)
        if token:
            try:
                similar_w2v = model_w2v.wv.most_similar(token, topn=5)
                print(f"  Word2Vec: {[w for w, s in similar_w2v]}")
            except:
                print(f"  Word2Vec: erreur")
        else:
            print(f"  Word2Vec: non trouvé")
        
        # Modèle 2
        try:
            similar_ft = model_fasttext_fr.wv.most_similar(word, topn=5)
            print(f"  FastText: {[w for w, s in similar_ft]}")
        except:
            print(f"  FastText: erreur")
else:
    print("\n⚠️ Modèle FastText non disponible (Partie 2 non exécutée ou erreur mémoire)")
    print("   Vous pouvez quand même analyser les résultats de votre modèle Word2Vec ci-dessus.")

### Questions de réflexion finale

Répondez à ces questions dans une cellule markdown ou dans un document séparé :

1. **Qualité des embeddings** : Votre modèle from scratch capture-t-il bien les relations Pokémon ? Donnez des exemples de similarités qui fonctionnent et d'autres qui échouent.

2. **Impact du tokenizer** : Comment le choix de la taille du vocabulaire BPE (8000 tokens) affecte-t-il les résultats ? Que se passerait-il avec 2000 ou 20000 tokens ?

3. **Limites du corpus** : Quels types de relations votre modèle ne peut PAS capturer ? (Indice : pensez aux informations absentes du corpus textuel)

4. **Comparaison** (si Partie 2 faite) : Dans quel cas préféreriez-vous le modèle from scratch ? Le modèle fine-tuné ? Justifiez.

5. **Applications** : Comment pourrait-on utiliser ces embeddings Pokémon dans une application réelle ? (ex: moteur de recherche, recommandation, chatbot...)

---

### Félicitations ! 🎉

Vous avez construit un **pipeline de représentation de texte** :

- ✅ Tokenizer BPE entraîné sur un corpus spécialisé
- ✅ Embeddings Word2Vec capturant la sémantique Pokémon
- ✅ Exploration des similarités et analogies
- ✅ (Bonus) Comparaison avec un modèle pré-entraîné

Ces techniques (tokenization subword + embeddings) sont les **briques de base** des modèles de langage modernes. GPT utilise BPE, et le concept d'embeddings appris est au cœur de tous les Transformers. Dans les prochains TPs, nous verrons comment le mécanisme d'**attention** permet de capturer le contexte !